### Business Task: Create a marketing strategy to increase the membership subscriptions. For this we need to check and find out the diffrences between Casual riders and Members from the data; So that we can make changes to marketing plan which will help us to convert Casual riders to Members.

#### Import essencial libraries for data processing, cleaning and visvalization

In [80]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
import cufflinks as cf
import plotly.graph_objs as go
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

#### As there are multiple data files writing a loop to read the file one by one and combining them at once
#### Observed that there are some abnormilities in the data in the month 12th of 2020. So, cleaned up the data by removing the problem rows before merging the data together (This ensures less data cleaning in next phase

In [81]:
frames=[]
a=[]
for i in ['04','05','06','07','08','09','10','11']:
    data=pd.read_csv('C:/Users/mrinamar/Desktop/G_project/data/2020-'+i+'.csv')
    frames.append(data)
for i in ['01','02','03','04']:
    data=pd.read_csv('C:/Users/mrinamar/Desktop/G_project/data/2021-'+i+'.csv')
    frames.append(data)
# Abnormilities in 12th month removed
data=pd.read_csv('C:/Users/mrinamar/Desktop/G_project/data/2020-12.csv')
data=data.drop(data[data['ended_at'].str.contains('^2020-11.*')==True].index)
frames.append(data)
mem_data=pd.concat(frames)

#### Dropping unnessasary data which have NA, the data which had missing values constitute less than 15%. Also, unnessasary columns .

In [82]:
mem_data=mem_data.dropna(subset=['start_station_name','end_station_name'])
mem_data=mem_data.drop(labels=['end_station_id','start_station_id','start_lat','start_lng','end_lat','end_lng'],axis=1)

In [83]:
mem_data.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
end_station_name      0
member_casual         0
dtype: int64

#### Inserting nessasary columns which are useful in futute analysis

In [84]:
mem_data.insert(4,'ride_length',((pd.to_datetime(mem_data['ended_at'])-pd.to_datetime(mem_data['started_at'])).dt.total_seconds()),True)
mem_data.insert(4,'year',pd.to_datetime(mem_data['started_at']).dt.year,True)
mem_data.insert(4,'month',pd.to_datetime(mem_data['started_at']).dt.month,True)
mem_data.insert(4,'day_of_week',(pd.to_datetime(mem_data['started_at'])).dt.weekday,True)

####  Summarizing the data to create a table with useful insights from the above table using gruopby()

In [85]:
trips_per_member=mem_data.groupby(['member_casual','year','month','day_of_week'],as_index=False).count().drop(labels=['ride_id','rideable_type','started_at','ended_at','start_station_name','end_station_name'],axis=1).rename(columns={'ride_length':'number_of_rides','member_casual':'subscriber','day_of_week':'day'})

trips_per_member.insert(4,'average_time_for_ride',mem_data.groupby(['member_casual','year','month','day_of_week'],as_index=False).mean().drop(labels=['member_casual','year','month','day_of_week'],axis=1),True)

trips_per_member.insert(4,'time_of_ride',mem_data.groupby(['member_casual','year','month','day_of_week'],as_index=False).sum().drop(labels=['member_casual','year','month','day_of_week'],axis=1),True)

In [86]:
trips_per_member

,subscriber,year,month,day,time_of_ride,average_time_for_ride,number_of_rides
0,casual,2020,4,0,9825535.0,3670.353007,2677
1,casual,2020,4,1,18407302.0,5041.715147,3651
2,casual,2020,4,2,6693281.0,3730.925864,1794
3,casual,2020,4,3,10559717.0,4352.727535,2426
4,casual,2020,4,4,16622608.0,6630.477862,2507
...,...,...,...,...,...,...,...
177,member,2021,4,2,17582714.0,780.274874,22534
178,member,2021,4,3,18988313.0,771.914021,24599
179,member,2021,4,4,25902310.0,820.134566,31583
180,member,2021,4,5,22615669.0,959.795824,23563


#### Descriptine analysis

In [87]:
print ('Mean of number of rides:',trips_per_member['number_of_rides'].mean())
print('===========================================================================')
print('Most rides by a Member:\n', trips_per_member.loc[trips_per_member['number_of_rides']==max(trips_per_member['number_of_rides'])])
print('===========================================================================')
print('Least number of rides by a Member:\n', trips_per_member.loc[trips_per_member['number_of_rides']==min(trips_per_member['number_of_rides'])])
print('===========================================================================')
print('Higest average munites of rides by a Member:\n', trips_per_member.loc[trips_per_member['average_time_for_ride']==max(trips_per_member['average_time_for_ride'])])
print('===========================================================================')
print('Least average munites of rides o rides by a Member:\n', trips_per_member.loc[trips_per_member['average_time_for_ride']==min(trips_per_member['average_time_for_ride'])])

Mean of number of rides: 19744.25824175824
Most rides by a Member:
    subscriber  year  month  day  time_of_ride  average_time_for_ride  \
33     casual  2020      8    5   202437724.0            2798.226885   

    number_of_rides  
33            72345  
Least number of rides by a Member:
    subscriber  year  month  day  time_of_ride  average_time_for_ride  \
70     casual  2021      2    0      853627.0            1880.235683   

    number_of_rides  
70              454  
Higest average munites of rides by a Member:
   subscriber  year  month  day  time_of_ride  average_time_for_ride  \
4     casual  2020      4    4    16622608.0            6630.477862   

   number_of_rides  
4             2507  
Least average munites of rides o rides by a Member:
     subscriber  year  month  day  time_of_ride  average_time_for_ride  \
154     member  2021      1    0     6662265.0             683.169094   

     number_of_rides  
154             9752  


#### Visvalizing the findings to make some sence of the data and to find some patterns. Using Plotly library to create area charts.

In [88]:
px.area(trips_per_member.groupby(['subscriber','month'],as_index=False).sum()[['number_of_rides','subscriber','month']],x='month',y='number_of_rides',width=800,height=400,title='Number of rides',facet_col='subscriber',color='subscriber')

In [89]:
px.area(trips_per_member.groupby(['subscriber','month'],as_index=False).sum()[['time_of_ride','subscriber','month']],x='month',y='time_of_ride',width=800,height=400,title='Time for rides',facet_col='subscriber',color='subscriber')

In [90]:
px.area(trips_per_member.groupby(['day','subscriber'],as_index=False).sum()[['day','number_of_rides','subscriber']],x='day',y='number_of_rides',color='subscriber',width=800,height=400,facet_col='subscriber')

In [91]:
px.area(trips_per_member.groupby(['day','subscriber'],as_index=False).sum()[['day','subscriber','time_of_ride']],x='day',y='time_of_ride',color='subscriber',width=800,height=400,facet_col='subscriber')

### Inferences:
* Number of rides taken by Members are high but time taken for rides are less and its the opposite for the Casual Riders.

* There is a large increase in the rides from 5th month to 10th month; This applies for both Casual Riders and Members. Inturn, the time taken for the rides also increased for both.

* Members have uniform number of rides through out the week, but Casual Riders take an increased number of rides on saturday. The same pattern is also seen for time of rides by both.

## Recommendations:

Based on the data collected we can onserve that,
* Casual members have take less nummber of rides but they travel large distances; So, in order to attract these coustomers there needs to be a benifit in the membership which befifits long distance rides.
* Also, Casual riders generally go out during saturday; So, offering some weekend discounts on the memberships might help.
* Focus on the months that have least rides and try aming to provide more discounts(as mentioned above) around that time will help in boosting the number of memberships.